 ФИО: Усцов Артем Алексеевич

In [ ]:
# uncomment to install pmdarima, plotly for the first
!pip install pmdarima
!pip install plotly

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import os
from tqdm import *
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pmdarima as pm
import numpy as np
%matplotlib inline

# Ноутбук для ДЗ №8 обработка временных последовательностей RNN
В данном дз вам будут данных времменая последовательно, которая описывает распределение хитов по времени за несколько лет. Вам нужно будет обучить модель RNN на исторических данных и потом сделать предсказание для "будущего" года, в текущей задаче это 2019 год

## Готовим данные
Три файла с данными
1. исторические данные - train
2. Тестовые данные текущего момента - derived
3. Пример сабмита резкльтатов конкурса на кагле

In [ ]:
# get data
!wget https://github.com/Totenkaf/DL_Homeworks/raw/main/HW_8/data/derived.csv -P data/
!wget https://github.com/Totenkaf/DL_Homeworks/raw/main/HW_8/data/sample_submission.csv -P data/
!wget https://github.com/Totenkaf/DL_Homeworks/raw/main/HW_8/data/train.csv -P data/

In [ ]:
def read_set(file):
    data = pd.read_csv(file)
    data['date'] = pd.to_datetime(data['date'])
    data = data.rename(columns = {'DATE':'date'})
    data = data.rename(columns = {'hits':'value'})
    data = data.set_index('date')
    return data

In [ ]:
data = pd.read_csv('data/train.csv')
data

In [ ]:
data_train = read_set('data/train.csv')
data_test =  read_set('data/derived.csv')
data_sample = read_set('data/sample_submission.csv')

In [ ]:
data_train

### Проверим наши данные, что мы загрузили

In [ ]:
print( data_train.info() )
print( data_test.info() )
print( data_sample.info() )

In [ ]:
data_train.describe()

In [ ]:
data_test.describe()

In [ ]:
data_sample.describe()

## Графики наших временных последовательностей

In [ ]:
plt.figure()
data_train['value'].plot(kind = 'line')
data_test['value'].plot(kind = 'line')
data_sample['value'].plot(kind = 'line')
plt.show()

## Статистическая модель [ARIMA](https://ru.wikipedia.org/wiki/ARIMA)


In [ ]:
# обучаем модель
arima_model=model = pm.auto_arima(data_train, seasonal = True,m = 4,test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)

In [ ]:
prediction = pd.DataFrame(model.predict(n_periods = int(data_test.size)), data_test.index)

In [ ]:
prediction

In [ ]:
prediction = prediction .rename(columns = {0:'value'})

смотрим, что она нам предсказала

In [ ]:
plt.figure()
data_train['value'].plot(kind = 'line')
data_test['value'].plot(kind = 'line')
#plt.plot(data_forecaste, label = "Prediction")
prediction['value'].plot(kind = 'line')
plt.show()

### Функция подсчета метрик для конкурса

In [ ]:
def MAPE(y_true, y_pred):
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), 1e-6)
    mape  = np.average(mape) * 100
    return mape

### MAPE для ARIMA и тестового сабмишена

In [ ]:
MAPE(data_test, prediction)

In [ ]:
MAPE(data_test, data_sample)

## Из пандас строим датасет

In [ ]:
class Stats:
    def __init__(self, dataset):
        self.mean = np.mean(dataset)
        self.std = np.std(dataset)
        self.data = (dataset - self.mean) / self.std 

stats = Stats(data_train)

In [ ]:
class TSDataset(Dataset):
    
    def __init__(self, data, seq_len):
        super().__init__()
        self._len = len(data) - seq_len + 1 # Кол-во проходов заданным окном
        self.mean = stats.mean 
        self.std = stats.std
        self.data = (data- self.mean) / self.std # Нормализация
        self.seq_len = seq_len # Длина окна
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, idx):
        d = self.data[idx:idx + self.seq_len] # Берем последовательность датафрейма
        targets = []
        days   = []
        months = []
        year = []
        weekday = []
        for row in  d.iterrows(): # итератор по строкам dataframe
            targets += [ row[1]['value'] ] # Получить value из строки
            days += [ row[0].day ] # 
            months += [row[0].month]
            year += [row[0].year]
            weekday += [row[0].weekday()]
            
        return torch.LongTensor(days), \
               torch.LongTensor(months), \
               torch.LongTensor(weekday), \
               torch.FloatTensor(targets)

In [ ]:
ds_train = TSDataset(data_train, 20) 
ds_test  = TSDataset(data_test, 20)
print(len(ds_train))


## Теперь нужно определить нашу модель 

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(777)

In [ ]:
import datetime
ans = datetime.date(2022, 1, 4)
ans.weekday()

In [ ]:
class TimeSeriesModel(nn.Module):
    def __init__(self, hidden_size: int, input_sizes: tuple):
        super().__init__()
        self.mon_emb = nn.Embedding(12+1, input_sizes[0]) # Эмбеддинги для месяцев (обучаемые)
        self.day_emb = nn.Embedding(31+1, input_sizes[1]) # Эмбеддинги для годов (обучаемые)
        self.weekday_emb = nn.Embedding(7, input_sizes[2])
        self.weekend_emb = nn.Embedding(7, input_sizes[3])
        
        # LSTM input: Эмбеддинги годов, месяцев, значение                      
        self._rnn = nn.LSTM(input_sizes[0] + input_sizes[1] + input_sizes[2] + input_sizes[3] + 1,
                            hidden_size,  batch_first=True, dropout=0.3)
        self._output = nn.Linear(hidden_size, 1)

    def forward(self, batch, ctx = None):
        days, mons, weekday, targets = batch
        mon_tensor = self.mon_emb(mons) # batch_sz x seq_len x emb_len (8 x 20 x 4)
        day_tensor = self.day_emb(days)
        weekday_tensor = self.weekday_emb(weekday)
        weekend = ((weekday == 5) | (weekday == 6))
        weekend_tensor = self.weekend_emb(weekend.long())
        rnn_input  = torch.cat([mon_tensor, day_tensor, weekday_tensor, weekend_tensor], dim=-1) # 8 x 20 x 8
        targets = targets.unsqueeze(-1)
        rnn_input = torch.cat([rnn_input, targets ], dim=-1) # 8 x 20 x 9
        # Берем все элементы последовательности, кроме последнего, предсказание идет на 1 шаг вперед
        rnn_input = rnn_input[:, :-1, :] if ctx is None else rnn_input # 8 x 19 x 9
        output, ctx = self._rnn(rnn_input, ctx)
        # 8 x 19 x 32, 1 x 8 x 32
        # print((self._output(output)).size()) 8 x 19 x 1
        output = self._output(output).squeeze() # 8 x 19 
        return output, ctx        

### Определяем даталоадеры для теста и трейна

In [ ]:
batch_sz = 8
hidden_size = 32
emb_size = (3, 4, 2, 1)

dl_train = DataLoader(ds_train, batch_sz , True)
dl_test = DataLoader(ds_test, batch_sz , False)
series_model = TimeSeriesModel(hidden_size, emb_size)

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
loss = nn.L1Loss()
optimizer = Adam(series_model.parameters(), lr=7e-4)# lr=5e-4)
scheduler = ExponentialLR(optimizer, gamma=0.97)

In [ ]:
# инициализируем тензорборд, для вывода графиков
writer = SummaryWriter(log_dir='./rnn_hw')

### Обучаем модель

In [ ]:
global_epoch = 0
global_iter = 0

In [ ]:
def test_model(epoch):
    test_iter  = tqdm(dl_test)
    sum_loss = 0
    num_batches = 0
    for i, batch in enumerate(test_iter):
        # Чтобы сохранялась временная зависимость
        # для предсказания таргет должен быть смешен на один временной шаг
        # относительно входа модели
        target = batch[-1][:, 1:]
        result, _ = series_model(batch)
        batch_loss = loss(result, target)
        sum_loss += batch_loss
        num_batches += 1
    sum_loss /= num_batches
    writer.add_scalar('Loss/val', sum_loss , epoch)
    print("Test:", sum_loss.item(), epoch)
    return sum_loss

In [ ]:
# модель обучаем в режиме teacher forcing, т.е. на вход подаем сразу всю последовательность,
# на выходе таргет должен быть смещен на один временной шаг, чтобы правильно считался лосс

for epoch in range(0, 500):
    epoch_iter = tqdm(dl_train)
    series_model.train()
    for batch in epoch_iter:
        optimizer.zero_grad()
        # Чтобы сохранялась временная зависимость
        # для предсказания таргет должен быть смешен на один временной шаг
        # относительно входа модели
        #print(batch[0].size())
        # batch.size() : 4 (day, month, year, target) x batch_sz x seg_len
        #import sys
        #sys.exit()
        target = batch[-1][:,1:] # Берем все значения, начиная  с 1
        result, hidden = series_model(batch)
        batch_loss = loss(result, target)
        batch_loss.backward()
        epoch_iter.set_description("Epoch: %04d, Iter Loss: %.4f"  %(epoch, batch_loss))
        writer.add_scalar('Loss/train', batch_loss , global_iter)
        global_iter += 1
        optimizer.step()
    scheduler.step()
    with torch.no_grad():
        series_model.eval()
        test_model(global_epoch)
    global_epoch += 1

In [ ]:
# сохраняем модель
torch.save(series_model.state_dict(), 'series_model2.ptx')

In [ ]:
# восстанавливаем модель
series_model = TimeSeriesModel(hidden_size, emb_size)
series_model.load_state_dict(torch.load('series_model2.ptx'))

### TODO
Теперь нам нужно для нашего тестового сета сгенерировать результат, и сделать правильный сабмишен. В отличии от режима обучения мы не должгны использовать значения таргетов в тесте, поэтому нам придется тут реализовывать инкрементальный режим генерации сети, т.е. когда на вход подаются фичи и таргет с предыдущего шага на каждоим шаге генерации.

In [ ]:
# новые даталоадеры НЕ перемешанные
new_ds_train = TSDataset(data_train, 1) 
new_ds_test  = TSDataset(data_test, 1)
new_dl_train = DataLoader(new_ds_train, 1 , False)
new_dl_test = DataLoader(new_ds_test, 1, False)

In [ ]:
# Уже с предобученной моделью - посчитаем h, c за все предыдущие года
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
hidden = (h, c)
ole = []
with torch.no_grad():
    series_model.eval()
    for batch in new_dl_train:
        result, hidden = series_model(batch, hidden) # Накапливаем память о последовательности
        ole.append(result.item())
last_res = result
results = []

with torch.no_grad():
    series_model.eval()
    for batch in new_dl_test:
        batch[3][0] = last_res
        results.append(last_res.item())
        last_res, hidden = series_model(batch, hidden)
results = np.array(results)

In [ ]:
plt.plot(results)

In [ ]:
results = results * stats.std.value + stats.mean.value

In [ ]:
prediction = pd.DataFrame(results, data_test.index)
prediction = prediction.rename(columns = {0:'value'})

In [ ]:
plt.figure()
data_train['value'].plot(kind = 'line')
data_test['value'].plot(kind = 'line')
#plt.plot(data_forecaste, label = "Prediction")
prediction['value'].plot()
plt.show()

In [ ]:
MAPE(data_test, prediction)

In [ ]:
MAPE(data_test, prediction)

In [ ]:
prediction = prediction.rename(columns = {"value" :'hits'})
prediction

In [ ]:
prediction.to_csv("submission.csv")